In [6]:
#!pip install torch==1.8.1 torchvision==0.4.1

In [7]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
import numpy as np
import os
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
torch.cuda.is_available()

True

In [3]:
def getGaussian(Nfft, Pk):
    xf = np.random.normal(0,1,Nfft)+1j*np.random.normal(0,1,Nfft)
    xf *= Pk
    return np.fft.irfft(xf)

def getNonGaussian(t):
    # Signal with non-Gaussian shape
    freq = np.random.uniform(200,500)
    phase = np.random.uniform(0,2*np.pi)
    sigma = 0.03
    pos = np.random.uniform(3*sigma,1-3*sigma)
    ampl = np.random.uniform (0.1,0.2)
    rfi = ampl*np.cos(phase+freq*t)*np.exp(-(t-pos)**2/(2*sigma**2))
    return rfi

def Gaussianize(signal):
    fsig = np.fft.rfft(signal)
    rot = np.exp(1j*np.random.uniform(0,2*np.pi,len(fsig)))
    return np.fft.irfft((fsig*rot))  

In [4]:
# define the network classes
# the decoder network
class Decoder(nn.Module):
  def __init__(self, z_dim, hidden_dim, out_dim):
    super(Decoder, self).__init__()
    self.main = nn.Sequential(
      nn.Linear(z_dim, hidden_dim),
      nn.LeakyReLU(0.02, inplace=False),

      nn.Linear(hidden_dim, hidden_dim * 2),
      nn.LeakyReLU(0.02, inplace=False),

      nn.Linear(hidden_dim * 2, out_dim, bias=False),
    )
    
  def forward(self, x):
    out = self.main(x)
    return out
    #return x-self.main(x)



# the encoder network
class Encoder(nn.Module):
  def __init__(self, input_dim, hidden_dim, z_dim):
    super(Encoder, self).__init__()

    self.main = nn.Sequential(
      nn.Linear(input_dim, hidden_dim * 2),
      nn.LeakyReLU(0.02, inplace=False),

      nn.Linear(hidden_dim * 2, hidden_dim),
      nn.LeakyReLU(0.02, inplace=False),
      nn.Dropout(0.2),

      nn.Linear(hidden_dim, z_dim)
    )
    
  def forward(self, x):
    out = self.main(x)
    return out
    #return x-self.main(x)

In [10]:
# training constants
BATCH_SIZE = 32
Z_DIM = 16
HIDDEN_DIM = 256
LR = 0.0002
EPOCHS = 3

# data parameters
N = 1024
Nfft = N//2+1
k = np.linspace(0,1,Nfft)
t = np.linspace(0,1,N)
Pk = (1+np.exp(-(k-0.5)**2/(2*0.1**2)))*np.exp(-k/0.5)

# train data
g_train_array = torch.stack([torch.from_numpy(getGaussian(Nfft, Pk)) for i in range(100000)])
ng_train_array = torch.stack([torch.from_numpy(getNonGaussian(t)) for i in range(100000)])

# test data
g_test_array = torch.stack([torch.from_numpy(getGaussian(Nfft, Pk)) for i in range(10)])
ng_test_array = torch.stack([torch.from_numpy(getNonGaussian(t)) for i in range(10)])

# create dataloaders
g_trainloader = DataLoader(
      torch.utils.data.TensorDataset(g_train_array),
      batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True, drop_last=True
)
ng_trainloader = DataLoader(
      torch.utils.data.TensorDataset(ng_train_array),
      batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True, drop_last=True
)

# initialize the networks
netD = Decoder(z_dim=Z_DIM, hidden_dim=HIDDEN_DIM, out_dim=N).cuda()
netE = Encoder(input_dim=N, hidden_dim=HIDDEN_DIM, z_dim=Z_DIM).cuda()

# define the optimizers
optimizer = optim.Adam([{'params': netE.parameters()}, 
                        {'params': netD.parameters()}], lr=LR, betas=(0.5, 0.999))

# loss criterion
recons_criterion = nn.MSELoss()

# training loop
iters = 0

for epoch in range(EPOCHS):
  # iterate through the dataloaders
  for i, (g, ng) in enumerate(zip(g_trainloader, ng_trainloader)):
      # set to train mode
      netE.train()
      netD.train()

      
      ng = ng[0].float().cuda()
      g = g[0].float().cuda()
      bs = ng.shape[0]

      # new inputs
      sigs = g + ng
      gaussianized = torch.stack([torch.from_numpy(Gaussianize(sig.cpu().numpy())) for sig in sigs]).cuda().float()
      modsig = sigs + gaussianized
      
      # for the noisy input get latent representation from the encoder
      z_out = netE(modsig)

      # decode the latent representation
      recons_out = netD(z_out)

      # compute the loss
      #loss = recons_criterion(recons_out, ng)
    
      loss = recons_criterion(modsig-recons_out, gaussianized)

      # backpropagate and update the weights
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      # print the training losses
      if iters % 100 == 0:
        print('[%3d/%d][%3d/%d]\tLoss: %.10f' 
            % (epoch, EPOCHS, i, len(g_trainloader), loss.item()))
      iters += 1    
  # visualize generated samples
  if True: ## end of every epoch
    # set to eval mode
    netD.eval()
    netE.eval()

    with torch.no_grad():
      # new inputs
      sigs = g_test_array.float().cuda() + ng_test_array.float().cuda()
      gaussianized = torch.stack([torch.from_numpy(Gaussianize(sig.cpu().numpy())) for sig in sigs]).cuda()
      modsig = (sigs + gaussianized).float()
      z_out = netE(modsig)
      recons_out = netD(z_out)

    # display a random sample from the test set
    rand_int = np.random.randint(10)

    # plot/save generated samples
    plt.figure(figsize=(17,10))
    ax = plt.subplot(2,3,1)
    plt.plot(t, ng_test_array[rand_int].cpu().numpy()) # ng
    ax.set_title("ng")

    ax = plt.subplot(2,3,2)
    plt.plot(t, g_test_array[rand_int].cpu().numpy()) # g
    ax.set_title("g")

    ax = plt.subplot(2,3,3)
    plt.plot(t, (g_test_array[rand_int].cpu() + ng_test_array[rand_int].cpu()).numpy()) # g + ng 
    ax.set_title("sig = ng + g")

    ax = plt.subplot(2,3,4)
    plt.plot(t, gaussianized[rand_int].cpu().numpy()) # gaussianize(ng + g)
    ax.set_title("Gauss(sig)")

    ax = plt.subplot(2,3,5)
    plt.plot(t, modsig[rand_int].cpu().numpy()) # input
    ax.set_title("network input=Gauss(sig) + sig")

    ax = plt.subplot(2,3,6)
    plt.plot(t, recons_out[rand_int].cpu().detach().numpy()) # output
    ax.set_title("network output (~ng)")

    save_folder = 'ae-g/'
    os.makedirs(save_folder, exist_ok=True)
    save_path = os.path.join(save_folder, str(epoch).zfill(7) + '.png')
    print('Saving file...{}'.format(save_path))
    plt.savefig(save_path, bbox_inches='tight')
    plt.close()
          


[  0/3][  0/3125]	Loss: 0.0021073672
[  0/3][100/3125]	Loss: 0.0013130317
[  0/3][200/3125]	Loss: 0.0012917498
[  0/3][300/3125]	Loss: 0.0012405853
[  0/3][400/3125]	Loss: 0.0012889009
[  0/3][500/3125]	Loss: 0.0012338536
[  0/3][600/3125]	Loss: 0.0012624802
[  0/3][700/3125]	Loss: 0.0012653806
[  0/3][800/3125]	Loss: 0.0013344057
[  0/3][900/3125]	Loss: 0.0012073349
[  0/3][1000/3125]	Loss: 0.0011862975
[  0/3][1100/3125]	Loss: 0.0012908929
[  0/3][1200/3125]	Loss: 0.0012190664
[  0/3][1300/3125]	Loss: 0.0012574324
[  0/3][1400/3125]	Loss: 0.0011578375
[  0/3][1500/3125]	Loss: 0.0012172877
[  0/3][1600/3125]	Loss: 0.0011953666
[  0/3][1700/3125]	Loss: 0.0011909356
[  0/3][1800/3125]	Loss: 0.0012198958
[  0/3][1900/3125]	Loss: 0.0012633000
[  0/3][2000/3125]	Loss: 0.0011285461
[  0/3][2100/3125]	Loss: 0.0012379328
[  0/3][2200/3125]	Loss: 0.0012443028
[  0/3][2300/3125]	Loss: 0.0011847841
[  0/3][2400/3125]	Loss: 0.0012259242
[  0/3][2500/3125]	Loss: 0.0012129752
[  0/3][2600/3125]	Los

In [12]:
from google.colab import files
#!zip -r /content/file.zip /content/ae-g
!zip -r file.zip /content/ae-g
#files.download("/content/file.zip")
files.download("file.zip")

ModuleNotFoundError: No module named 'google'